In [ ]:
# # 0) Réglages pour éviter la bagarre de threads sous-jacents
# import os, faulthandler, sys
# os.environ.update({
#     "PYTHONMALLOC": "debug",              # hooks Python sur alloc
#     "MALLOC_CHECK_": "3",                 # vérifs glibc malloc (héritage)
#     "GLIBC_TUNABLES": "glibc.malloc.check=3",  # vérifs glibc récentes
#     "OMP_NUM_THREADS": "1",               # pas de bagarre OpenMP/BLAS
#     "OPENBLAS_NUM_THREADS": "1",
#     "MKL_NUM_THREADS": "1",
#     "NUMEXPR_NUM_THREADS": "1",
#     "VECLIB_MAXIMUM_THREADS": "1",
# })

# faulthandler.enable()

import os, sys

print("Python:", sys.version)
N_cpu = os.cpu_count() # 8
print(N_cpu)
# os.environ["OMP_NUM_THREADS"] = "1"       # au cas où miniball ou BLAS threadent
# os.environ["OPENBLAS_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# os.environ["NUMEXPR_NUM_THREADS"] = "1"

Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
8


In [ ]:
# # !pip install gudhi
# # Colab / Jupyter
# # %pip uninstall -y gudhi numpy scipy
# # %pip install --no-cache-dir --force-reinstall "numpy" "gudhi=3.10.*" "scipy>=1.14" "pybind11>=2.12"
# # Installe GUDHI 3.11.0 proprement dans Colab
# %pip uninstall -y gudhi numpy -q
# %pip install -q --no-cache-dir --force-reinstall --only-binary=:all: \
#   "numpy==2.0.*" "gudhi==3.11.0"

# import sys, numpy as np, gudhi
# print("Python:", sys.version.split()[0],
#       "| NumPy:", np.__version__,
#       "| GUDHI:", gudhi.__version__)
# import gudhi as gd
# print(gd.__debug_info__)
# print(gd.__available_modules)

In [ ]:
%load_ext cython
%config Cython.annotate = True
%config Cython.language_level = 3
%reload_ext cython

In [ ]:
%%cython
# cython: boundscheck=False
# cython: nonecheck=False
# cython: initializedcheck=False
# Tree handling (condensing, finding stable clusters) for hdbscan
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION

import numpy as np
cimport numpy as np

cdef np.double_t INFTY = np.inf

cdef class UnionFind:
    cdef np.intp_t[:] parent
    cdef np.intp_t[:] _size
    def __init__(self, int n):
        self.parent = np.arange(n, dtype=np.intp)
        self._size  = np.ones(n, dtype=np.intp)
    cpdef int find(self, int x):
        cdef int r=x
        while self.parent[r]!=r:
            r=self.parent[r]
        cdef int cur = x
        cdef int nxt
        while self.parent[cur]!=r:
            nxt=self.parent[cur]
            self.parent[cur]=r
            cur=nxt
        return r
    cpdef bint union(self,int x,int y):
        cdef int rx=self.find(x)
        cdef int ry=self.find(y)
        if rx==ry: return False
        if self._size[rx]<self._size[ry]:
            self.parent[rx]=ry; self._size[ry]+=self._size[rx]
        else:
            self.parent[ry]=rx; self._size[rx]+=self._size[ry]
        return True
    cpdef int component_size(self,int x):
        return self._size[self.find(x)]



ctypedef np.double_t DTYPE_t
ctypedef np.int64_t  ITYPE_t

# Calcule (q, w) pour un k-uplet d'indices, avec s2_all = ||p_i||^2 pré-calculé.
# w = ||q||^2 - mean(s2_all[idx])
cpdef double bary_weight_one(DTYPE_t[:, ::1] M, DTYPE_t[::1] s2_all,
                             ITYPE_t[::1] idx, DTYPE_t[::1] out_q):
    cdef Py_ssize_t k = idx.shape[0]
    cdef Py_ssize_t d = M.shape[1]
    cdef Py_ssize_t i, t
    cdef double smean = 0.0
    cdef double qnorm2 = 0.0
    cdef ITYPE_t ii

    # accumuler les sommes des coordonnées
    for t in range(d):
        out_q[t] = 0.0

    for i in range(k):
        ii = idx[i]
        smean += s2_all[ii]
        for t in range(d):
            out_q[t] += M[ii, t]

    # moyenne et poids
    for t in range(d):
        out_q[t] /= k
        qnorm2 += out_q[t] * out_q[t]

    smean /= k
    return qnorm2 - smean


# Version batch: combos (m,k) -> Q (m,d) et w (m,)
cpdef void bary_weight_batch(DTYPE_t[:, ::1] M, DTYPE_t[::1] s2_all,
                             ITYPE_t[:, ::1] combos,
                             DTYPE_t[:, ::1] out_Q, DTYPE_t[::1] out_w):
    cdef Py_ssize_t m = combos.shape[0]
    cdef Py_ssize_t k = combos.shape[1]
    cdef Py_ssize_t d = M.shape[1]
    cdef Py_ssize_t i, j, t
    cdef double smean, qnorm2
    cdef ITYPE_t ii

    for i in range(m):
        smean = 0.0
        # reset ligne de Q
        for t in range(d):
            out_Q[i, t] = 0.0
        # accumulateurs
        for j in range(k):
            ii = combos[i, j]
            smean += s2_all[ii]
            for t in range(d):
                out_Q[i, t] += M[ii, t]
        # moyenne des coordonnées
        for t in range(d):
            out_Q[i, t] /= k
        # poids
        smean /= k
        qnorm2 = 0.0
        for t in range(d):
            qnorm2 += out_Q[i, t] * out_Q[i, t]
        out_w[i] = qnorm2 - smean


cpdef int union_if_adjacent_int(ITYPE_t[::1] a, ITYPE_t[::1] b, ITYPE_t[::1] out_u):
    cdef Py_ssize_t k = a.shape[0]
    cdef Py_ssize_t i = 0
    cdef Py_ssize_t j = 0
    cdef Py_ssize_t u = 0

    while i < k and j < k:
        if u >= out_u.shape[0]:
            return 0
        if a[i] == b[j]:
            out_u[u] = a[i]; i += 1; j += 1; u += 1
        elif a[i] < b[j]:
            out_u[u] = a[i]; i += 1; u += 1
        else:
            out_u[u] = b[j]; j += 1; u += 1

    while i < k:
        if u >= out_u.shape[0]:
            return 0
        out_u[u] = a[i]; i += 1; u += 1

    while j < k:
        if u >= out_u.shape[0]:
            return 0
        out_u[u] = b[j]; j += 1; u += 1

    return 1 if u == k + 1 else 0




cdef inline np.int64_t _min_i64(np.int64_t a, np.int64_t b) nogil:
    return a if a <= b else b

cdef inline np.int64_t _max_i64(np.int64_t a, np.int64_t b) nogil:
    return a if a >= b else b

cpdef tuple build_leaf_dfs_intervals(np.ndarray[np.int64_t, ndim=1] left,
                                      np.ndarray[np.int64_t, ndim=1] right):
    """
    Retourne:
      pos        (m,)     int64  leaf_id -> rang DFS
      first,last (m+t,)   int64  intervalle de feuilles par nœud [first,last]
      leaf_order (m,)     int64  rang DFS -> leaf_id
    Hypothèse SciPy: parent i = m+i, enfants < parent.
    """
    cdef Py_ssize_t t = left.shape[0]
    if right.shape[0] != t:
        raise ValueError("left/right must have same length")
    cdef Py_ssize_t m = t + 1
    cdef Py_ssize_t n_nodes = m + t

    cdef np.int64_t[:] L = left
    cdef np.int64_t[:] R = right

    cdef np.ndarray[np.int64_t, ndim=1] first      = np.empty(n_nodes, dtype=np.int64)
    cdef np.ndarray[np.int64_t, ndim=1] last       = np.empty(n_nodes, dtype=np.int64)
    cdef np.ndarray[np.int64_t, ndim=1] leaf_order = np.empty(m,       dtype=np.int64)
    cdef np.ndarray[np.int64_t, ndim=1] pos        = np.empty(m,       dtype=np.int64)

    cdef np.int64_t[:] first_v = first
    cdef np.int64_t[:] last_v  = last
    cdef np.int64_t[:] lo_v    = leaf_order
    cdef np.int64_t[:] pos_v   = pos

    cdef Py_ssize_t i
    for i in range(n_nodes):
        first_v[i] = -1
        last_v[i]  = -1

    # Pile itérative: (node, state) ; state=0 pre, 1 post
    cdef np.ndarray[np.int64_t, ndim=1] stack_node = np.empty(n_nodes, dtype=np.int64)
    cdef np.ndarray[np.int8_t,  ndim=1] stack_st   = np.empty(n_nodes, dtype=np.int8)
    cdef np.int64_t[:] st_node = stack_node
    cdef np.int8_t[:]  st_st   = stack_st

    cdef Py_ssize_t sp = 0
    cdef np.int64_t root = m + t - 1
    st_node[sp] = root; st_st[sp] = 0; sp += 1

    cdef Py_ssize_t k = 0
    cdef np.int64_t x, state, child_idx, a, b, fa, fb, la, lb

    while sp > 0:
        sp -= 1
        x = st_node[sp]
        state = st_st[sp]

        if x < m:
            first_v[x] = k
            last_v[x]  = k
            lo_v[k]    = x
            k += 1
            continue

        child_idx = x - m
        if not (0 <= child_idx < t):
            raise ValueError("Invalid internal node index")

        if state == 0:
            # post-ordre: on empile soi (state=1), puis les enfants
            st_node[sp] = x; st_st[sp] = 1; sp += 1
            b = R[child_idx]
            a = L[child_idx]
            if a >= x or b >= x or a < 0 or b < 0:
                raise ValueError("SciPy linkage convention violated: child >= parent")
            st_node[sp] = b; st_st[sp] = 0; sp += 1
            st_node[sp] = a; st_st[sp] = 0; sp += 1
        else:
            a = L[child_idx]
            b = R[child_idx]
            fa = first_v[a]; fb = first_v[b]
            la = last_v[a];  lb = last_v[b]
            if fa == -1 or fb == -1:
                raise ValueError("Invalid tree: child interval not computed")
            first_v[x] = _min_i64(fa, fb)
            last_v[x]  = _max_i64(la, lb)

    if k != m:
        raise ValueError("Leaf DFS did not visit all leaves")

    for i in range(m):
        pos_v[lo_v[i]] = i

    return pos, first, last, leaf_order

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=False)

# #%run /content/drive/MyDrive/Colab_modules/custom_hdbscan.ipynb

# import os, sys, pyximport
# sys.path.append('/content/drive/MyDrive/Colab_modules')

# # Compiler à l'import, et mettre les binaires dans Drive pour les réutiliser
# build_dir = '/content/drive/MyDrive/Colab_modules/_build'
# os.makedirs(build_dir, exist_ok=True)

# pyximport.install(
#     language_level=3,
#     build_dir=build_dir,
#     setup_args={"include_dirs": [np.get_include()]}  # nécessaire si cimport numpy
# )

# import custom_hdbscan  # compile si nécessaire, sinon réutilise le .so déjà construit

In [ ]:
# from custom_hdbscan import condense_tree, compute_stability, get_clusters

import hdbscan
from hdbscan._hdbscan_tree import condense_tree, compute_stability, get_clusters

def tree_to_labels(single_linkage_tree, min_cluster_size=20, DBSCAN_threshold=None, cluster_selection_method="eom", allow_single_cluster=True, match_reference_implementation=False, cluster_selection_epsilon=0.0, cluster_selection_persistence=0.0, max_cluster_size=0, cluster_selection_epsilon_max=float('inf')):
    """Converts a pretrained tree and cluster size into a
    set of labels and probabilities.
    """
    condensed_tree = condense_tree(single_linkage_tree, min_cluster_size)
    stability_dict = compute_stability(condensed_tree)
    # if single_linkage_tree.shape[0] > threshold_tree_size_for_EoM :
    #     print("Z.shape[0] =",single_linkage_tree.shape[0],">",threshold_tree_size_for_EoM," => cluster_selection_method='leaf'")
    #     labels, probabilities, stabilities = get_clusters(condensed_tree, stability_dict, "leaf", allow_single_cluster, match_reference_implementation, cluster_selection_epsilon, max_cluster_size, cluster_selection_epsilon_max)
    # else :
    if DBSCAN_threshold is None :
        labels, probabilities, stabilities = get_clusters(condensed_tree, stability_dict, cluster_selection_method, allow_single_cluster, match_reference_implementation, cluster_selection_epsilon, max_cluster_size, cluster_selection_epsilon_max)
    else :
        # print("DBSCAN_threshold = ", DBSCAN_threshold)
        labels, probabilities, stabilities = get_clusters(condensed_tree, stability_dict, "leaf", allow_single_cluster, match_reference_implementation, DBSCAN_threshold, max_cluster_size, DBSCAN_threshold)
    return (labels, probabilities, stabilities, condensed_tree, single_linkage_tree)

/usr/local/lib/python3.12/dist-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
import itertools
import scipy.spatial

convex_hull = scipy.spatial.ConvexHull # convex_hull_from_gudhi_delaunay #

class Cell:
    # list of vertices of the cell, which themselves are k-tuples of points
    # k: the k for which this cell appears
    def __init__(self, vertices, k):
        self.vertices = vertices
        self.k = k

    def __str__(self):
        return str(self.vertices) + '[%d]' % self.k


class OrderKDelaunay:
    """Order-k Delaunay mosaic for a set of points up to a given order k.

    The constructor takes a list of input points (with a point represented as
    a list of its coordinates) and an order. The point set can be in
    Euclidean space of any dimension. Upon construction, the order-k Delaunay
    mosaics of the points set from order 1 up to the specified order are
    computed. The result can be accessed via the public attributes of the
    OrderKDelaunay instance.

    Public attributes:
        diagrams_vertices:
            List of vertex lists.
            For each order-k Delaunay mosaic from 1 up to order, the list
            of vertices where each vertex is a k-tuple of point indices.
        diagrams_simplices:
            List of simplex lists.
            For each order-k Delaunay mosaic from 1 up to order,
            the list of top-dimensional simplices of a triangulation of the
            mosaic, where each simplex is a (d+1)-tuple of indices into the
            corresponding vertex list.
        diagrams_cells:
            List of cell lists.
            For each order-k Delaunay mosaic from 1 up to order,
            the list of top-dimensional cells of the mosaic, where each cell
            is a tuple of vertices (i.e. k-tuples of point indices) which spans
            the cell (i.e. the convex hull is convex hull of the vertices).
        diagrams_generations:
            List of generation lists.
            For each order-k Delaunay mosaic from 1 up to order,
            the list of generations of the top-dimensional cells of the mosaic,
            i.e. each cell from diagrams_cells is associated a generation.
    """

    def __init__(self, points, order, verbeux=False):
        '''
        Parameters:
            points - list of points
            order - order k up to which to compute the order-k Delaunay mosaics
        '''
        self.diagrams_vertices = []
        self.diagrams_simplices = []
        self.diagrams_cells = []
        self.diagrams_generations = []
        self.verbeux=verbeux

        # Dimension of the ambient space.
        self._dimension = len(points[0])
        # Store each point with its magnitude as last coordinate, giving a
        # a point in R^d+1, because we're using a lower convex hull to get
        # the order-k cells from the vertex set.
        self.lifts = [tuple(np.append(c, np.linalg.norm(c)**2))
                      for c in points]

        # Compute all the mosaics
        self._compute_order_1()
        for k in range(2, order + 1):
            self._compute_order_k(k)

    def _compute_order_1(self):
        # Get first order Delaunay mosaic as lower convex hull of the lifts
        chull = convex_hull(self.lifts) # scipy.spatial.ConvexHull(self.lifts) #, qhull_options='Qs')
        if self.verbeux :
            print("_compute_order_1 done")
        # chull.equations[i][dimension] < 0 means only taking the
        # lower convex hull of the lifts
        simplices = [chull.simplices[i] for i in range(len(chull.simplices))
                     if chull.equations[i][self._dimension] < 0]

        tupled_simplices = [[(vertex,) for vertex in simplex]
                            for simplex in simplices]
        self.diagrams_vertices.append([(i,) for i in range(len(self.lifts))])
        self.diagrams_simplices.append(simplices)
        self.diagrams_cells.append(tupled_simplices)
        self.diagrams_generations.append([1] * len(simplices))
        # Make a list of the order 1 cells.
        # cell_queue will be used as the list of cells who haven't gone through
        # all their barycentric polytopes yet.
        self.cell_queue = list(zip(tupled_simplices, [1] * len(simplices)))
        self.cell_queue = [Cell(*cell) for cell in self.cell_queue]

    def _compute_order_k(self, k):
        # Step 2.1: Compute the vertices and generation >= 2 cells of the
        # order-k Delaunay mosaic.
        new_nextgen_cells = []
        new_generations = []
        new_vertices = []
        cell_queue_new = []
        # Go over all cells whose cycle of barycentric polytopes we haven't
        # completed yet.
        for cell in self.cell_queue:
            # k - cell.k is the generation of the new barycentric cell we get.
            generation = k - cell.k
            if generation < self._dimension:
                # Take all (generation+1)-tuples of vertices
                # --> list of gtuples of vertices (which are tuples of points)
                gtuples = list(itertools.combinations(
                      cell.vertices, generation + 1))
                # For each of these tuples of vertices, we take the union of
                # the vertices to get a new vertex.
                # The outer 'sorted' is not necessary, but ensures output is
                # entirely sorted, which is useful for testing and comparing.
                nvs = sorted([tuple(sorted(set.union(*[set(vertex)
                      for vertex in gtuple]))) for gtuple in gtuples])
                # The number of points in each new vertex should be k+1,
                # otherwise something went wrong.
                new_nextgen_cells.append(nvs)
                new_generations.append(generation + 1)
                new_vertices += nvs
            else:
                pass

            # Once the generation is dimension - 1, we've cycled through all
            # barycentric polytopes and thus don't need to add the cell to the
            # queue again
            if k - cell.k < self._dimension - 1:
                cell_queue_new.append(cell)

        # List of new vertices without repetitions.
        new_vertices = list(set(new_vertices))
        # For each tuple that we identified as vertex,
        # compute the centroid of its lifts.
        new_lifts = np.array([np.sum(
              [self.lifts[i] for i in new_vertex], axis=0) / k
                    for new_vertex in new_vertices])

        chull = convex_hull(new_lifts) # scipy.spatial.ConvexHull(new_lifts) #, qhull_options='Qs')
        if self.verbeux :
            print("_compute_order_k done, k=", k)
        # Compute the simplices of the triangulated order-k Delaunay mosaic,
        # which is the lower convex hull of these centroids.
        # Each simplex is a tuple of integers, these integers are indices into
        # new_vertices, which contains the k-tuples of original points which
        # are vertices.
        # (chull.equations[i][dimension] < 0 means only taking the
        # lower convex hull of the lifts.)
        simplices = [sorted(chull.simplices[i])
              for i in range(len(chull.simplices))
                    if chull.equations[i][self._dimension] < 0]

        # Step 2.2: Compute the remaining cells of the order-k Delaunay mosaic
        new_firstgen_cells = []
        for simplex in simplices:
            # Get the k-tuples that are the vertices of the simplex.
            vertices = [set(new_vertices[i]) for i in simplex]
            x_in = set.intersection(*vertices)
            # Simplices are first generation if the intersection of their
            # vertices is k-1.
            if len(x_in) == k - 1:
                # The outer 'sorted' is not necessary, but ensures the output
                # is entirely sorted, which is useful for testing and comparing
                cell_vertices = sorted([new_vertices[i] for i in simplex])
                new_firstgen_cells.append(cell_vertices)
                cell_queue_new.append(Cell(cell_vertices, k))

        # Use our compiled queue for the next iteration
        self.cell_queue = cell_queue_new

        # Save the computed stuff
        self.diagrams_vertices.append(new_vertices)
        self.diagrams_simplices.append(simplices)
        self.diagrams_cells.append(new_nextgen_cells + new_firstgen_cells)
        self.diagrams_generations.append(
            new_generations + [1] * len(new_firstgen_cells))



# import matplotlib.colors as colors
# import matplotlib.pyplot as plt
# import mpl_toolkits.mplot3d as a3
# import pylab

# class Plotter:
#     '''Abstract class for plotting order-k Delaunay mosaics.

#     The actual plotting functionality is implemented in Plotter2D and
#     Plotter3D. All classes use the constructor from Plotter, which
#     defines the following public attributes for plotting settings:

#     Public attributes:
#         draw_input_points: True/False - Whether to draw the input point set.
#         draw_labels: True/False - Whether to draw labels for the vertices and
#                      input points.
#         colors_cells: list of 3 matplotlib.colors - for first, second and
#                       third generation cells.
#     '''

#     def __init__(self, points, orderk_delaunay):
#         '''Initialize various drawing settings.

#         These are public attributes and can be modified by the user.
#         '''
#         self._points = points
#         self._orderk_delaunay = orderk_delaunay

#         # Whether to draw a label for each vertex
#         self.draw_labels = True

#         # Whether to draw a label for each vertex
#         self.draw_input_points = True

#         # colors of the cells of each generation
#         color_firstgen_cell = colors.rgb2hex((1.0, 0.0, 0.0))
#         color_secondgen_cell = colors.rgb2hex((0.0, 0.0, 1.0))
#         color_thirdgen_cell = colors.rgb2hex((1.0, 1.0, 0.0))
#         self.colors_cells = [
#               color_firstgen_cell, color_secondgen_cell, color_thirdgen_cell]

#         # separator of the points indices that the vertex label consists of
#         self._label_sep = ''
#         if len(points) > 10:
#             self._label_sep = ','

#     def draw(self, order):
#         '''Stub. Implemented by subclasses.'''
#         pass


# class Plotter2D(Plotter):

#     def __init__(self, points, orderk_delaunay):
#         super().__init__(points, orderk_delaunay)

#     def draw(self, order):
#         vertices = self._orderk_delaunay.diagrams_vertices[order-1]
#         cells = self._orderk_delaunay.diagrams_cells[order-1]
#         generations = self._orderk_delaunay.diagrams_generations[order-1]

#         ax = plt.gca()
#         ax.cla()
#         # Draw cells stemming from barycentric subdivisions of older cells.
#         # Either new_nextgen_cells or triangulated_cells is empty, depending on
#         # whether 'triangulate' is True or False.
#         for tri, gen in zip(cells, generations):
#             vxs = []
#             for vx in tri:
#                 xs = [self._points[c][0] for c in vx]
#                 ys = [self._points[c][1] for c in vx]
#                 xcenter = sum(xs)/len(xs)
#                 ycenter = sum(ys)/len(ys)
#                 vxs.append([xcenter, ycenter])
#             p = plt.Polygon(vxs, closed=True, fill=True,
#                             color=self.colors_cells[gen-1], alpha = 0.4)
#             ax.add_patch(p)
#             for pair in itertools.combinations(vxs, 2):
#                 pair = np.array(pair)
#                 line = plt.Line2D(pair[:, 0], pair[:, 1],
#                       color=self.colors_cells[gen-1], linewidth = 1.0)
#                 ax.add_artist(line)

#         # order-k points
#         centroids = np.array([np.sum(
#               [self._points[i] for i in vertex], axis=0)/order
#                     for vertex in vertices])
#         ax.plot(centroids[:,0], centroids[:,1], 'o', color="black")
#         if self.draw_labels:
#             for c, v in zip(centroids, vertices):
#                 ax.text(c[0], c[1], self._label_sep.join([str(pt) for pt in v]))
#         # first-order points
#         if self.draw_input_points:
#             ax.plot(self._points[:,0], self._points[:,1], 'o', color='red')
#             if self.draw_labels:
#                 for i in range(len(self._points)):
#                     ax.text(self._points[i][0], self._points[i][1], str(i))

#         plt.show()


# class Plotter3D(Plotter):

#     def __init__(self, points, orderk_delaunay):
#         super().__init__(points, orderk_delaunay)

#         # Factor by how much to shrink simplices (helps visualization).
#         self.shrinking_factor = 0.25

#     def draw(self, order):
#         vertices = self._orderk_delaunay.diagrams_vertices[order-1]
#         cells = self._orderk_delaunay.diagrams_cells[order-1]
#         generations = self._orderk_delaunay.diagrams_generations[order-1]

#         ax = a3.Axes3D(pylab.figure())
#         for cell, gen in zip(cells, generations):
#             gvertices = []
#             # compute the geometric vertices
#             for gvertex in cell:
#                 gvertices.append(np.mean(np.array(
#                       [self._points[i][0:3] for i in gvertex]), axis=0))
#             gvertices = np.array(gvertices)
#             col = self.colors_cells[gen-1]
#             faces = []
#             cell_center = np.mean(gvertices, axis=0)

#             # tetrahedron
#             if len(cell) == 4:
#                 for facet in itertools.combinations(gvertices, 3):
#                     # geometric facet
#                     gfacet = [gv[0:3]*(1-self.shrinking_factor) + \
#                           cell_center*(self.shrinking_factor) for gv in facet]
#                     faces.append(gfacet)
#             # octahedron
#             elif len(cell) == 6:
#                 for facet_ids in itertools.combinations(range(len(gvertices)), 3):
#                     facet = [cell[i] for i in facet_ids]
#                     gfacet = [gvertices[i] for i in facet_ids]
#                     is_facet = True
#                     # three vertices only span a facet if they pairwise differ
#                     # in only one element.
#                     for edge in itertools.combinations(facet, 2):
#                         if len(set(edge[0]) & set(edge[1])) != order-1:
#                             is_facet = False
#                     if is_facet:
#                         # geometric facet
#                         gfacet = [gv[0:3]*(1-self.shrinking_factor) + \
#                               cell_center*(self.shrinking_factor)
#                                     for gv in gfacet]
#                         faces.append(gfacet)
#             tri = a3.art3d.Poly3DCollection(faces, alpha=0.2)
#             tri.set_color(col)
#             tri.set_edgecolor('k')
#             ax.add_collection3d(tri)
#         # order-k points
#         centroids = np.array([np.sum(
#               [self._points[i] for i in vertex], axis=0)/order
#                     for vertex in vertices])
#         ax.scatter(centroids[:,0], centroids[:,1], centroids[:,2], 'o',
#                    color="black")
#         if self.draw_labels:
#             for c, v in zip(centroids, vertices):
#                 ax.text(c[0], c[1], c[2], self._label_sep.join([str(pt) for pt in v]))
#         # first-order points
#         if self.draw_input_points:
#             ax.scatter(self._points[:,0], self._points[:,1], self._points[:,2],
#                   marker='o',color='red')
#             if self.draw_labels:
#                 for i in range(len(self._points)):
#                     ax.text(self._points[i][0], self._points[i][1],
#                             self._points[i][2], str(i))

#         plt.show()

In [ ]:
# # # 2D example point set
# # points = np.array([[156.006,705.854],
# #         [215.257,732.63], [283.108,707.272], [244.042,670.948],
# #         [366.035,687.396], [331.768,625.715], [337.936,559.92],
# #         [249.525,582.537], [187.638,556.13], [165.912,631.197]])

# # 3D example point set
# # points = np.array([(0,0,0), (0,4,4), (4,4,0), (4,0,4), (-10,2,2)])
# rng = np.random.default_rng(7)
# points = rng.random((10, 2))

# # the order k up to which to compute the order-k Delaunay diagram
# order = 3
# # Whether to print the cells of all the complexes
# print_output = True
# # Whether to draw all the order-k Delaunay mosaics
# draw_output = True

# # Compute the order-k Delaunay mosaics
# orderk_delaunay = OrderKDelaunay(points, order)

# # Initialize appropriate plotter for drawing the mosaics.
# dimension = len(points[0])
# if dimension == 2:
#     plotter = Plotter2D(points, orderk_delaunay)
# elif dimension == 3:
#     plotter = Plotter3D(points, orderk_delaunay)
# else:
#     # Stub that doesn't draw anything.
#     plotter = Plotter(points, orderk_delaunay)

# for k in range(1, order+1):
#     if draw_output:
#         plotter.draw(k)

#     if print_output:
#         cells = orderk_delaunay.diagrams_cells[k-1]
#         # Output all the cells.
#         print("Order {}. Number of cells: {}".format(
#                 len(cells[0][0]), len(cells)))
#         for cell in sorted(cells):
#             print(cell)

In [ ]:
# !nvidia-smi
# !nvcc --version

In [ ]:
# !apt-get update
# !apt-get install -y build-essential git cmake nvidia-cuda-toolkit

In [ ]:
# !git clone https://github.com/Ludwig-H/gDel3D.git
# %cd gDel3D
# !cmake -S . -B build
# !cmake --build build -j8

In [ ]:
!sudo apt-get update
!sudo apt-get install -y build-essential cmake libcgal-dev libtbb-dev libtbbmalloc2 libgmp-dev libmpfr-dev libeigen3-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [81.0 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,307 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [5,804 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:14 h

In [ ]:
!git clone https://github.com/Ludwig-H/EdgesCGALWeightedDelaunay3D.git
%cd EdgesCGALWeightedDelaunay3D
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j
%cd ..

Cloning into 'EdgesCGALWeightedDelaunay3D'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 31.66 KiB | 900.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/EdgesCGALWeightedDelaunay3D
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGALConfig.cmake:92 (message):
  CGAL_DATA_DIR cannot be deduced, set the variable CGAL_DATA_DIR to set the
  default value of CGAL::data_file_path()
Call Stack (most recent call first):
  CMakeLists.txt:15 (find_package)


-- Using header-only CGAL
-- Targetting Unix Makefiles
-- Using /usr/bin/c

In [ ]:
!git clone https://github.com/Ludwig-H/EdgesCGALWeightedDelaunay2D.git
%cd EdgesCGALWeightedDelaunay2D
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j
%cd ..

Cloning into 'EdgesCGALWeightedDelaunay2D'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 27 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 24.42 KiB | 735.00 KiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/EdgesCGALWeightedDelaunay2D
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGALConfig.cmake:92 (message):
  CGAL_DATA_DIR cannot be deduced, set the variable CGAL_DATA_DIR to set the
  default value of CGAL::data_file_path()
Call Stack (most recent call first):
  CMakeLists.txt:6 (find_package)


-- Using header-only CGAL
-- Targetting Unix Makefiles
-- Using /usr/bin/c+

In [ ]:
!git clone https://github.com/Ludwig-H/EdgesCGALWeightedDelaunayND.git
%cd EdgesCGALWeightedDelaunayND
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j
%cd ..

Cloning into 'EdgesCGALWeightedDelaunayND'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 25 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 28.42 KiB | 786.00 KiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/EdgesCGALWeightedDelaunayND
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGALConfig.cmake:92 (message):
  CGAL_DATA_DIR cannot be deduced, set the variable CGAL_DATA_DIR to set the
  default value of CGAL::data_file_path()
Call Stack (most recent call first):
  CMakeLists.txt:7 (find_package)


-- Using header-only CGAL
-- Targetting Unix Makefiles
-- Using /usr/bin/c+

In [ ]:
!git clone https://github.com/Ludwig-H/EdgesCGALDelaunay3D.git
%cd EdgesCGALDelaunay3D
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j
%cd ..

Cloning into 'EdgesCGALDelaunay3D'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 25.64 KiB | 729.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/EdgesCGALDelaunay3D
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGALConfig.cmake:92 (message):
  CGAL_DATA_DIR cannot be deduced, set the variable CGAL_DATA_DIR to set the
  default value of CGAL::data_file_path()
Call Stack (most recent call first):
  CMakeLists.txt:14 (find_package)


-- Using header-only CGAL
-- Targetting Unix Makefiles
-- Using /usr/bin/c++ compiler.
-- 

In [ ]:
!git clone https://github.com/Ludwig-H/EdgesCGALDelaunay2D.git
%cd EdgesCGALDelaunay2D
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j
%cd ..

Cloning into 'EdgesCGALDelaunay2D'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 10), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 16.11 KiB | 471.00 KiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/EdgesCGALDelaunay2D
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGALConfig.cmake:92 (message):
  CGAL_DATA_DIR cannot be deduced, set the variable CGAL_DATA_DIR to set the
  default value of CGAL::data_file_path()
Call Stack (most recent call first):
  CMakeLists.txt:6 (find_package)


-- Using header-only CGAL
-- Targetting Unix Makefiles
-- Using /usr/bin/c++ compiler.
-

In [ ]:
!git clone https://github.com/Ludwig-H/EdgesCGALDelaunayND.git
%cd EdgesCGALDelaunayND
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release
!cmake --build build -j
%cd ..

Cloning into 'EdgesCGALDelaunayND'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 36.60 KiB | 961.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/EdgesCGALDelaunayND
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGALConfig.cmake:92 (message):
  CGAL_DATA_DIR cannot be deduced, set the variable CGAL_DATA_DIR to set the
  default value of CGAL::data_file_path()
Call Stack (most recent call first):
  CMakeLists.txt:12 (find_package)


-- Using header-only CGAL
-- Targetting Unix Makefiles
-- Using /usr/bin/c++ compiler.
-- 

In [ ]:
!pip install gudhi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 24.8 MB/s eta 0:00:00


In [ ]:
# import gudhi
from typing import List



def _unique_sorted_rows(arr: np.ndarray, *, sort_rows: bool = False) -> np.ndarray:
    """
    Déduplique des lignes d'entiers en supprimant les doublons (ensemble = ligne),
    et renvoie les lignes triées lexicographiquement.
    - Forçage dtype int64
    - Optionnellement trie chaque ligne (utile si l'amont ne garantit pas ordre croissant)
    - Tri global lexicographique
    - Filtre les doublons adjacents
    - Retourne un tableau C-contigu

    Paramètres
    ----------
    arr : (m, k) array-like d'entiers

    Retour
    ------
    (u, k) np.ndarray[int64]
    """
    a = np.asarray(arr, dtype=np.int64)
    if a.ndim != 2:
        raise ValueError(f"_unique_sorted_rows attend un 2D array, reçu {a.shape} / ndim={a.ndim}")
    m, k = a.shape
    if m == 0:
        return a.reshape(0, k)

    # # S'assurer que chaque ligne est strictement croissante si nécessaire
    if sort_rows:
        # mergesort est stable; pas vital ici, mais ça garde un comportement prévisible
        a = np.sort(a, axis=1, kind="mergesort")

        # Tri lexicographique des lignes: dernière colonne = clé primaire
        # a.T[::-1] produit la liste des clés de lexsort dans le bon ordre
        order = np.lexsort(a.T[::-1])
        s = np.ascontiguousarray(a[order])  # C-contigu pour la suite
    else :
        s = np.ascontiguousarray(a)

    # Garde la première ligne de chaque run de lignes identiques
    keep = np.ones(m, dtype=bool)
    if m > 1:
        keep[1:] = np.any(s[1:] != s[:-1], axis=1)

    return s[keep]



def _edges_from_weighted_delaunay(points: np.ndarray, weights: np.ndarray = None, precision: str = "safe") -> np.ndarray:
    """
    Graphe (1-squelette) de la Delaunay pondérée, pour points en R^d.
    Retourne un tableau (e, 2) int64 avec i<j, dédoublonné.
    """
    import subprocess
    # w_points = np.column_stack((points, weights))   # shape (n, 4)
    N,d = points.shape
    root_CGAL = "/content"
    input_file = './my_point_cloud'+str(N)+'.npy'
    input_file_weights = './my_weights'+str(N)+'.npy'
    output_file = "./sortie"+str(N)+".npy"
    np.save(input_file, points)
    if weights is not None:
        np.save(input_file_weights, weights)
    if d == 3 :
        if weights is not None:
            command = [root_CGAL+"/EdgesCGALWeightedDelaunay3D/build/EdgesCGALWeightedDelaunay3D", input_file, input_file_weights, output_file]
        else :
            command = [root_CGAL+"/EdgesCGALDelaunay3D/build/EdgesCGALDelaunay3D", input_file, output_file]
    elif d == 2 :
        if weights is not None:
            command = [root_CGAL+"/EdgesCGALWeightedDelaunay2D/build/EdgesCGALWeightedDelaunay2D", input_file, input_file_weights, output_file]
        else :
            command = [root_CGAL+"/EdgesCGALDelaunay2D/build/EdgesCGALDelaunay2D", input_file, output_file]
    else :
        if weights is not None :
            command = [root_CGAL+"/EdgesCGALWeightedDelaunayND/build/EdgesCGALWeightedDelaunayND", input_file, input_file_weights, output_file]
        else :
            command = [root_CGAL+"/EdgesCGALDelaunayND/build/EdgesCGALDelaunayND", input_file, output_file]
    result = subprocess.run(command, capture_output=True, text=True, check=True)

    edges_list = np.load(output_file).tolist()

    os.remove(input_file)
    if weights is not None:
        os.remove(input_file_weights)
    os.remove(output_file)

    return edges_list



def _build_all_keys(combos: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Aplati toutes les clés (k-1)-aires de tous les k-uplets.
    combos: (m,k) int64 trié par ligne.
    Retourne:
      keys:    (m*k, k-1) int64  — toutes les clés, une par (combo,position)
      parents: (m*k,)     int64  — index du combo parent pour chaque clé
    """
    m, k = combos.shape
    if k < 2:
        return np.empty((0, 0), dtype=np.int64), np.empty((0,), dtype=np.int64)
    keys_blocks = []
    parents_blocks = []
    rng = np.arange(m, dtype=np.int64)
    for r in range(k):
        # garder toutes colonnes sauf r
        keep = [c for c in range(k) if c != r]
        keys_blocks.append(combos[:, keep])
        parents_blocks.append(rng)  # chaque ligne de ce bloc vient de combo i
    keys = np.vstack(keys_blocks).astype(np.int64, copy=False)
    parents = np.concatenate(parents_blocks).astype(np.int64, copy=False)
    return keys, parents

# def orderk_delaunay3(M: np.ndarray, K: int, precision: str = "safe", verbeux:bool = False) -> list[list[int]]:
#     """
#     Variante locale optimisée: une seule passe de tri pour regrouper par toutes les clés (k-1)-aires.
#     - K=1: Delaunay non pondérée sur M, retourne les arêtes [i,j].
#     - K>=2: barycentres/poids pour tous les k-uplets, puis pour chaque clé (unique)
#             on triangule localement une seule fois et on forme les (k+1)-uplets adjacents.
#     """
#     M = np.asarray(M, dtype=np.float64)
#     if M.ndim != 2 or M.shape[1] < 1:
#         raise ValueError(f"M doit être (n,d) avec d>=1, reçu {M.shape}")
#     if K < 1:
#         raise ValueError("K doit être >= 1")
#     n = M.shape[0]
#     if n < 2:
#         return []

#     # Pré-calcul ||p_i||^2
#     s2_all = (M * M).sum(axis=1)

#     # K=1: graphe Delaunay (poids nuls)
#     edges = _edges_from_weighted_delaunay(M, np.zeros(n, dtype=np.float64), precision)
#     prev = edges  # (m,2) int64

#     if verbeux :
#         print("orderk_delaunay k = 1")

#     if K == 1:
#         return prev.tolist()

#     for k in range(2, K + 1):
#         if prev.shape[0] < 2:
#             return []

#         # k-uplets uniques
#         combos = _unique_sorted_rows(prev.astype(np.int64, copy=False))
#         combos = np.ascontiguousarray(combos, dtype=np.int64)

#         # Barycentres/poids pour tous les combos
#         Q = np.empty((combos.shape[0], M.shape[1]), dtype=np.float64)
#         w = np.empty((combos.shape[0],), dtype=np.float64)
#         bary_weight_batch(M, s2_all, combos, Q, w)

#         # Une seule passe: construire toutes les clés (k-1)-aires et les trier une fois
#         keys, parents = _build_all_keys(combos)
#         if keys.size == 0:
#             return []

#         # Tri lexicographique des clés
#         key_cols = [keys[:, c] for c in range(keys.shape[1] - 1, -1, -1)]
#         order = np.lexsort(key_cols)
#         keys_sorted = keys[order]
#         parents_sorted = parents[order]

#         # Délimitation des runs de clés égales
#         start = np.ones(keys_sorted.shape[0], dtype=bool)
#         start[1:] = np.any(keys_sorted[1:] != keys_sorted[:-1], axis=1)
#         run_starts = np.flatnonzero(start)
#         run_ends = np.r_[run_starts[1:], keys_sorted.shape[0]]

#         next_candidates = []
#         out_u = np.empty(k + 1, dtype=np.int64)

#         # Une triangulation pondérée par clé unique
#         for a, b in zip(run_starts, run_ends):
#             L = b - a
#             if L < 2:
#                 continue
#             loc = parents_sorted[a:b]                  # indices des combos dans cette étoile
#             # loc = np.unique(loc)                       # au cas où
#             if loc.size < 2:
#                 continue

#             e_local = _edges_from_weighted_delaunay(Q[loc], w[loc], precision)
#             if e_local.size == 0:
#                 continue

#             for i, j in e_local:
#                 A = combos[loc[i]]
#                 B = combos[loc[j]]
#                 if union_if_adjacent_int(A, B, out_u):
#                     next_candidates.append(out_u.copy())

#         if not next_candidates:
#             return []

#         prev = _unique_sorted_rows(np.asarray(next_candidates, dtype=np.int64))
#         prev = np.ascontiguousarray(prev, dtype=np.int64)
#         if verbeux :
#             print(f"orderk_delaunay k = {k}")

#     return prev.tolist()

# def _sparsify_with_indices(Q: np.ndarray, epsilon: float,
#                            *, rtol: float = 0.0, atol: float = 0.0) -> tuple[np.ndarray, np.ndarray]:
#     """
#     Appelle gudhi.subsampling.sparsify_point_set(Q, epsilon) qui NE renvoie pas les indices.
#     On reconstruit les indices en supposant que l’ordre des points est préservé.
#     Repli robuste si le scan échoue (doublons / allclose).
#     Retourne (Q_sub, idx) où idx indexe Q.
#     """
#     if epsilon <= 0.0 or Q.shape[0] < 3:
#         return Q, np.arange(Q.shape[0], dtype=np.int64)

#     # Gudhi attend une liste Python
#     Q_sub_list = gudhi.subsampling.sparsify_point_set(Q.tolist(), min_squared_dist=epsilon**2)
#     if len(Q_sub_list) == 0:
#         return Q[:0], np.empty((0,), dtype=np.int64)

#     Q_sub = np.asarray(Q_sub_list, dtype=np.float64)
#     # 1) Chemin rapide: deux-pointeurs, égalité stricte, ordre préservé
#     idx = []
#     j = 0
#     for i in range(Q.shape[0]):
#         if j >= Q_sub.shape[0]:
#             break
#         if np.array_equal(Q[i], Q_sub[j]):
#             idx.append(i)
#             j += 1
#     if j == Q_sub.shape[0]:
#         return Q[idx], np.asarray(idx, dtype=np.int64)

#     # 2) Repli tolérant: deux-pointeurs avec allclose
#     idx = []
#     j = 0
#     for i in range(Q.shape[0]):
#         if j >= Q_sub.shape[0]:
#             break
#         if np.allclose(Q[i], Q_sub[j], rtol=rtol, atol=atol):
#             idx.append(i)
#             j += 1
#     if j == Q_sub.shape[0]:
#         return Q[idx], np.asarray(idx, dtype=np.int64)

#     # 3) Repli robuste: dictionnaire de listes de positions (gère les doublons)
#     #    On quantize pour clés stables si besoin.
#     key = tuple
#     def _key(v: np.ndarray) -> tuple:
#         return tuple(v.tolist())

#     pos = {}
#     for i in range(Q.shape[0]):
#         k = _key(Q[i])
#         pos.setdefault(k, []).append(i)
#     idx = []
#     for row in Q_sub:
#         k = _key(np.asarray(row, dtype=np.float64))
#         if k not in pos or not pos[k]:
#             raise RuntimeError("Impossible de reconstituer les indices après sparsification. "
#                                "Hypothèse d'ordre préservé probablement violée.")
#         idx.append(pos[k].pop(0))
#     idx = np.asarray(idx, dtype=np.int64)
#     return Q[idx], idx


def orderk_delaunay3(M: np.ndarray, K: int, precision: str = "safe", verbeux: bool = False) -> list[list[int]]:
    """
    Version globale avec sparsification (Gudhi) avant chaque Delaunay pondérée.
    Échantillonnage contrôlé par epsilon *= ||quantile_95(M) - quantile_05(M)||.

      - K=1 : Delaunay non pondérée sur M → arêtes [i,j].
      - K>=2 : calcule (barycentres, poids) pour TOUS les k-uplets,
               SPARSIFIE (Q, w) et 'combos' de manière cohérente,
               Delaunay pondérée globale sur l'échantillon,
               puis ne garde que les unions de taille k+1.

    Retour: liste de k-uplets (indices int) triés.
    """
    M = np.ascontiguousarray(M, dtype=np.float64)
    if M.ndim != 2 or M.shape[1] < 1:
        raise ValueError(f"M doit être (n,d) avec d>=1, reçu {M.shape}")
    if K < 1:
        raise ValueError("K doit être >= 1")
    n, d = M.shape
    if n < 2:
        return []

    # Pré-calcul ||p_i||^2 (option 2 que tu as retenue)
    s2_all = (M * M).sum(axis=1)

    # # Échelle epsilon basée sur M: ||q95 - q05||
    # q05 = np.quantile(M, 0.05, axis=0)
    # q95 = np.quantile(M, 0.95, axis=0)
    # epsilon = 0.000001*float(np.linalg.norm(q95 - q05))
    # if verbeux:
    #     print(f"epsilon (sparsify) = {epsilon:.6g}")

    # K=1 : graphe Delaunay non pondéré
    prev = _edges_from_weighted_delaunay(M, precision=precision)
    if verbeux:
        print("orderk_delaunay k = 1")
    if K == 1:
        return prev

    for k in range(2, K + 1):
        if len(prev) < 2:
            return []

        # k-uplets uniques triés
        # combos = _unique_sorted_rows(np.ascontiguousarray(prev, dtype=np.int64), sort_rows=False)
        combos_s = np.ascontiguousarray(prev, dtype=np.int64)
        # Barycentres/poids pour tous les combos (batch Cython recommandé)
        Q_s = np.empty((combos_s.shape[0], d), dtype=np.float64)
        w_s = np.empty((combos_s.shape[0],), dtype=np.float64)
        bary_weight_batch(M, s2_all, combos_s, Q_s, w_s)
        if verbeux :
            print("Nouveaux barycentres pondérés calculés.")

        # # Sparsification (sans indices → on les reconstruit)
        # if epsilon > 0.0 and combos.shape[0] > 2:
        #     Q_s, idx = _sparsify_with_indices(Q, epsilon)
        #     if idx.size >= 2:
        #         combos_s = combos[idx]
        #         w_s = w[idx]
        #     else:
        #         combos_s, Q_s, w_s = combos, Q, w
        # else:
        #     combos_s, Q_s, w_s = combos, Q, w

        if verbeux :
            print(f"Q_s.shape[0] = {Q_s.shape[0]}")


        # Delaunay pondérée globale sur l’échantillon
        e_global = _edges_from_weighted_delaunay(Q_s, w_s, precision=precision)
        if verbeux :
            print("e_global : ", len(e_global))
        if len(e_global) == 0:
            return []

        # Unions k→k+1 (adjacences d'ordre k)
        next_candidates = []
        out_u = np.empty(k + 1, dtype=np.int64)
        for i, j in e_global:
            A = combos_s[i]
            B = combos_s[j]
            if union_if_adjacent_int(A, B, out_u):
                next_candidates.append(out_u.copy())

        if not next_candidates:
            return []

        if verbeux :
            print("next_candidates : ", len(next_candidates))
        prev = _unique_sorted_rows(np.asarray(next_candidates, dtype=np.int64), sort_rows=False)
        if verbeux :
            print("prev : ", prev.shape)
        prev = np.ascontiguousarray(prev, dtype=np.int64)
        if verbeux:
            print(f"orderk_delaunay k = {k}")

    return prev.tolist()

In [ ]:
# ==== Optimized v7: bucketing edges per component, vectorized chains, selectable distinct_mode ====

def _kruskal_mst_from_edges(n_nodes, rows, cols, weights, UF):
    rows = np.asarray(rows, dtype=np.int64)
    cols = np.asarray(cols, dtype=np.int64)
    weights = np.asarray(weights, dtype=float)
    if rows.size == 0:
        return []
    order = np.argsort(weights, kind="stable")
    rows = rows[order]; cols = cols[order]; weights = weights[order]
    mst_u = np.empty(max(0, n_nodes - 1), dtype=np.int64)
    mst_v = np.empty_like(mst_u)
    mst_w = np.empty_like(mst_u, dtype=float)
    taken = 0
    for i in range(rows.size):
        a = int(rows[i]); b = int(cols[i])
        ra = UF.find(a); rb = UF.find(b)
        if ra == rb:
            continue
        mst_u[taken] = a; mst_v[taken] = b; mst_w[taken] = float(weights[i])
        UF.union(ra, rb)
        taken += 1
        if taken == mst_u.size:
            break
    return list(zip(mst_u[:taken].tolist(), mst_v[:taken].tolist(), mst_w[:taken].tolist()))

def build_Z_mst_occurrences_gc(face_vertices,
                               mst_faces_sorted,   # [(u,v,w)] ou (u_arr, v_arr, w_arr)
                               min_cluster_size,
                               verbose=False,
                               distinct_mode="owner",
                               DBSCAN_threshold=None):

    def _fmt_bytes(n):
        n = float(n)
        if n < 1024: return f"{n:.0f} B"
        n /= 1024
        if n < 1024: return f"{n:.1f} KiB"
        n /= 1024
        if n < 1024: return f"{n:.1f} MiB"
        n /= 1024
        return f"{n:.2f} GiB"

    def _rss():
        try:
            import psutil
            return psutil.Process().memory_info().rss
        except Exception:
            return None

    # Faces compactes
    F = np.asarray(face_vertices, dtype=np.int64, order="C")
    m = int(F.shape[0])
    K = int(F.shape[1])
    if m == 0:
        return np.zeros((0,4), np.float64), np.zeros(0, np.int64), F

    if verbose:
        r = _rss()
        print(f"[F-GC:0] faces={m}, K={K}, faces_bytes={_fmt_bytes(F.nbytes)}, RSS={_fmt_bytes(r) if r else 'n/a'}")

    # Arêtes du MST des faces -> arrays + tri par poids
    if isinstance(mst_faces_sorted, tuple) and len(mst_faces_sorted)==3:
        u_arr, v_arr, w_arr = mst_faces_sorted
        u_arr = np.asarray(u_arr, np.int64)
        v_arr = np.asarray(v_arr, np.int64)
        w_arr = np.asarray(w_arr, np.float64)
    else:
        u_arr = np.fromiter((u for u,_,_ in mst_faces_sorted), count=len(mst_faces_sorted), dtype=np.int64)
        v_arr = np.fromiter((v for _,v,_ in mst_faces_sorted), count=len(mst_faces_sorted), dtype=np.int64)
        w_arr = np.fromiter((w for _,_,w in mst_faces_sorted), count=len(mst_faces_sorted), dtype=np.float64)

    if u_arr.size == 0:
        return np.zeros((0,4), np.float64), np.zeros(m, np.int64), F

    order = np.argsort(w_arr, kind="stable")
    u_arr = u_arr[order]; v_arr = v_arr[order]; w_arr = w_arr[order]

    if verbose:
        r = _rss()
        print(f"[F-GC:1] MST-face edges={u_arr.size}, edges_bytes≈{_fmt_bytes(u_arr.nbytes+v_arr.nbytes+w_arr.nbytes)}, RSS={_fmt_bytes(r) if r else 'n/a'}")

    # Structures pour Kruskal au niveau des faces
    next_face = np.full(m, -1, dtype=np.int64)
    head = np.arange(m, dtype=np.int64)
    tail = np.arange(m, dtype=np.int64)
    comp_sz = np.ones(m, dtype=np.int64)

    UF = UnionFind(m)
    cid = np.arange(m, dtype=np.int64)
    nxt = int(m)

    # Owner des sommets (global vertex ids) pour compter "distincts"
    n_points = int(F.max()) + 1
    owner_root = np.full(n_points, -1, dtype=np.int64)
    distinct_count = np.zeros(m, dtype=np.int64)

    # Pré-distribution: chaque sommet est d’abord "owned" par la première face qui le voit
    # et on crédite cette face d’1 distinct. O(m*K) mais c’est linéaire et cache-friendly.
    for f in range(m):
        row = F[f]
        for j in range(K):
            v = int(row[j])
            if owner_root[v] == -1:
                owner_root[v] = f
                distinct_count[f] += 1

    # Hiérarchie au niveau des faces (Z with mixed dtypes)
    Z = np.empty((max(0, m-1),4), dtype=np.float64)#[('cluster_id_big', np.intp),('cluster_id_small', np.intp),('weight', np.float64),('distinct_count', np.intp)])
    taken = 0

    if verbose:
        alloc = (next_face.nbytes + head.nbytes + tail.nbytes + comp_sz.nbytes +
                 cid.nbytes + owner_root.nbytes + distinct_count.nbytes + Z.nbytes)
        r = _rss()
        print(f"[F-GC:2] alloc aux≈{_fmt_bytes(alloc)}, RSS={_fmt_bytes(r) if r else 'n/a'}")

    # Kruskal streaming sur les arêtes du MST des faces
    for a, b, w in zip(u_arr, v_arr, w_arr):
        ra = UF.find(int(a)); rb = UF.find(int(b))
        if ra == rb:
            continue
        sa = int(comp_sz[ra]); sb = int(comp_sz[rb])
        if sa < sb: small, big = ra, rb
        else:       small, big = rb, ra

        root_big = UF.find(big)

        # Compte exact des distincts en scannant la petite composante (faces -> sommets)
        uniq_add = 0
        i = int(head[small])
        while i != -1:
            row = F[i]
            for j in range(K):
                v = int(row[j])
                pr = owner_root[v]
                # Si ce sommet n’appartient pas déjà à la composante 'big' (post-trouvée), on le bascule et on incrémente
                if pr == -1 or UF.find(int(pr)) != root_big:
                    owner_root[v] = root_big
                    uniq_add += 1
            i = int(next_face[i])

        new_count = int(distinct_count[root_big]) + int(uniq_add)

        # Écrit une ligne de Z: [id_big, id_small, poids, distincts]
        Z[taken,0] = float(cid[big])
        Z[taken,1] = float(cid[small])
        Z[taken,2] = float(w)
        Z[taken,3] = float(new_count)
        taken += 1

        # Concatène les listes de faces small -> big
        if head[small] != -1:
            if head[big] == -1:
                head[big] = head[small]; tail[big] = tail[small]
            else:
                next_face[int(tail[big])] = int(head[small])
                tail[big] = int(tail[small])
            comp_sz[big] = comp_sz[big] + comp_sz[small]
            head[small] = -1; tail[small] = -1; comp_sz[small] = 0

        # Union et propagation des méta-infos au nouveau root
        UF.union(big, small)
        root = UF.find(big)
        head[root] = head[big]; tail[root] = tail[big]; comp_sz[root] = comp_sz[big]
        distinct_count[root] = new_count
        cid[root] = nxt; nxt += 1

        if verbose and (taken % 5_000_000 == 0):
            r = _rss()
            print(f"[F-GC:merge] merges={taken}/{m-1}, RSS={_fmt_bytes(r) if r else 'n/a'}")

        if taken == m - 1:
            break

    Z = Z if taken == Z.shape[0] else Z[:taken].copy()

    if verbose:
        r = _rss()
        print(f"[F-GC:done] Z(shape={Z.shape}, dtype={Z.dtype}), RSS={_fmt_bytes(r) if r else 'n/a'}")

    Z_pruned, surv_idx = prune_linkage_by_inclusion(Z_full=Z, K=K, verbose=verbose)
    if verbose:
        r = _rss()
        print(f"[F-GC:Tree2Links] Z_pruned.shape[0]={Z_pruned.shape[0]} RSS={_fmt_bytes(r) if r else 'n/a'}")

    if Z_pruned.shape[0] <= 1 or Z_pruned[-1,3] <= min_cluster_size :
        return Z_pruned.shape, np.full(m, -2, dtype=np.int64), np.zeros(m, np.float64), F

    # Clustering sur les FACES (labels par face)
    labels_faces, probabilities_faces, *_ = tree_to_labels(
        single_linkage_tree=Z_pruned,
        min_cluster_size=min_cluster_size,
        DBSCAN_threshold=DBSCAN_threshold
    )
    if verbose:
        r = _rss()
        print(f"[F-GC: Tree2Labels]  RSS={_fmt_bytes(r) if r else 'n/a'}")

    # On renvoie: Z, labels PAR FACE, et la table face_vertices compacte
    ret_labels = np.full(m, -2, dtype=np.int64)
    ret_prob = np.zeros(m, dtype=np.float64)
    if verbose :
        print(len(surv_idx), len(ret_labels),len(ret_labels[surv_idx]),len(labels_faces))
    ret_labels[surv_idx] = labels_faces
    ret_prob[surv_idx] = probabilities_faces
    return Z, ret_labels, ret_prob, F



# def prune_linkage_by_inclusion(Z_full: np.ndarray, K: int, verbose: bool=False):
#     """
#     Émonde Z_full par inclusion et reconstruit un linkage SciPy VALIDE uniquement
#     sur les feuilles survivantes. La colonne 3 (taille d'union) est reprise EXACTEMENT
#     depuis Z_full pour chaque fusion conservée.

#     Hypothèses SciPy:
#       - m = Z_full.shape[0] + 1 feuilles
#       - parents en ordre: parent_i = m + i, et Z[i,0], Z[i,1] < m + i
#       - Z[i] = [childL, childR, weight, union_size] avec union_size = |A ∪ B| (“distincts”)
#       - chaque feuille vaut implicitement K (utile seulement pour tester l’inclusion au premier niveau)
#     """
#     Z = np.asarray(Z_full, dtype=np.float64, order="C")
#     t = int(Z.shape[0])
#     m = t + 1
#     if m <= 1:
#         return np.zeros((0,4), np.float64), np.arange(m, dtype=np.int64)

#     left   = Z[:, 0].astype(np.int64,   copy=False)
#     right  = Z[:, 1].astype(np.int64,   copy=False)
#     weight = Z[:, 2].astype(np.float64, copy=False)
#     usize  = Z[:, 3].astype(np.int64,   copy=False)

#     n_nodes = m + t

#     # Tailles de nœuds pour tester l'inclusion: feuille=K, interne=usize
#     size_node = np.zeros(n_nodes, dtype=np.int64)
#     size_node[:m]   = K
#     size_node[m:]   = usize

#     # 1) Élagage par inclusion: DFS sur le "loser" (aucune liste partagée)
#     alive_leaf = np.ones(m, dtype=bool)
#     dead_node  = np.zeros(n_nodes, dtype=bool)

#     def kill_subtree(root: int):
#         stack = [int(root)]
#         while stack:
#             x = stack.pop()
#             if dead_node[x]:
#                 continue
#             dead_node[x] = True
#             if x < m:
#                 alive_leaf[x] = False
#             else:
#                 i = x - m
#                 stack.append(int(left[i]))
#                 stack.append(int(right[i]))

#     for i in range(t):
#         a = int(left[i]); b = int(right[i]); p = m + i
#         sa = int(size_node[a]); sb = int(size_node[b]); su = int(size_node[p])
#         # inclusion ssi union == max(sa, sb) → on supprime TOUT le sous-arbre perdant
#         if su == (sa if sa >= sb else sb):
#             loser = b if sa >= sb else a
#             kill_subtree(loser)

#     # 2) Marque "alive" pour tous les nœuds (pas que les feuilles)
#     alive_node = np.zeros(n_nodes, dtype=bool)
#     alive_node[:m] = alive_leaf
#     for i in range(t-1, -1, -1):
#         a = int(left[i]); b = int(right[i]); p = m + i
#         alive_node[p] = alive_node[a] | alive_node[b]

#     # 3) Réindexation et reconstruction en une passe
#     #    new_id[x] = id dans l’arbre PRUNÉ, sinon -1
#     new_id = np.full(n_nodes, -1, dtype=np.int64)
#     # feuilles survivantes → 0..L-1
#     surv_idx = np.nonzero(alive_leaf)[0].astype(np.int64)
#     L = int(surv_idx.size)
#     if L <= 1:
#         return np.zeros((0,4), np.float64), surv_idx
#     new_id[surv_idx] = np.arange(L, dtype=np.int64)

#     # sorties
#     Z_pruned = np.empty((L-1, 4), dtype=np.float64)
#     created = 0

#     for i in range(t):
#         a = int(left[i]); b = int(right[i]); p = m + i
#         na = int(new_id[a]); nb = int(new_id[b])

#         if na == -1 and nb == -1:
#             # aucun survivant sous ce parent: tout est mort → parent s’effondre
#             new_id[p] = -1
#             continue

#         if na == -1 or nb == -1:
#             # un seul côté vivant: on "raccroche" en se rétractant sur le côté vivant
#             new_id[p] = nb if na == -1 else na
#             continue

#         # deux côtés vivants: on GARDE la fusion
#         ida, idb = (na, nb) if na < nb else (nb, na)  # ordre esthétique
#         parent_new = L + created

#         Z_pruned[created, 0] = float(ida)
#         Z_pruned[created, 1] = float(idb)
#         Z_pruned[created, 2] = float(weight[i])
#         Z_pruned[created, 3] = float(usize[i])   # on garde exactement la taille d’union d’origine

#         new_id[p] = parent_new
#         created += 1
#         if created == L - 1:
#             break

#     if created != L - 1:
#         # Si ça arrive, c’est que Z_full viole la convention SciPy (ou a été corrompu).
#         raise RuntimeError(f"[PRUNE] Quotient incomplet: L={L}, edges={created}, attendu {L-1}")

#     if verbose:
#         print(f"[PRUNE] feuilles: {m} → survivantes: {L} | Z_pruned: {Z_pruned.shape}")

#     return Z_pruned, surv_idx

# # Helper: construit l’ordre DFS des feuilles et les intervalles [first,last] de feuilles pour chaque nœud
# def build_leaf_dfs_intervals(left: np.ndarray, right: np.ndarray):
#     """
#     Retourne:
#       pos        : (m,) int64, mapping leaf_id -> position DFS [0..m-1]
#       first,last : (m+t,) int64, bornes d’intervalle de feuilles pour chaque nœud (0..m+t-1)
#       leaf_order : (m,) int64, inverse de pos (leaf_id_by_pos)
#     Hypothèse SciPy: parent_i = m+i, enfants < parent.
#     """
#     t = int(left.size)
#     m = t + 1
#     n_nodes = m + t

#     first = np.full(n_nodes, -1, dtype=np.int64)
#     last  = np.full(n_nodes, -1, dtype=np.int64)
#     leaf_order = np.empty(m, dtype=np.int64)

#     root = m + t - 1
#     stack = [(int(root), 0)]
#     k = 0
#     while stack:
#         x, state = stack.pop()
#         if x < m:
#             first[x] = last[x] = k
#             leaf_order[k] = x
#             k += 1
#         else:
#             i = x - m
#             if state == 0:
#                 stack.append((x, 1))
#                 stack.append((int(right[i]), 0))
#                 stack.append((int(left[i]), 0))
#             else:
#                 a = int(left[i]); b = int(right[i])
#                 fa, fb = int(first[a]), int(first[b])
#                 la, lb = int(last[a]),  int(last[b])
#                 if fa == -1 or fb == -1:
#                     raise RuntimeError("Arbre invalide: intervalle enfant manquant.")
#                 first[x] = fa if fa <= fb else fb
#                 last[x]  = la if la >= lb else lb

#     if k != m:
#         raise RuntimeError("DFS feuilles incomplet.")
#     pos = np.empty(m, dtype=np.int64)
#     pos[leaf_order] = np.arange(m, dtype=np.int64)
#     return pos, first, last, leaf_order


def prune_linkage_by_inclusion(Z_full: np.ndarray, K: int, verbose: bool=False):
    """
    Option 2 robuste (compression par inclusion) compatible avec ta UnionFind Cython (find/union).

    Étapes:
      1) Calcule les intervalles de feuilles [first,last] pour chaque nœud via DFS.
      2) Marque les fusions inclusives (su == max(sa,sb)) et contracte TOUTE la branche perdante
         en unionnant l’intervalle de feuilles perdant dans la classe de l’intervalle gagnant.
      3) Construit un graphe d’arêtes entre classes pour chaque fusion non inclusive.
      4) Kruskal sur les classes → Z_pruned de taille (L'−1,4), avec la 3e colonne copiée depuis Z_full.
      5) Renvoie aussi surv_idx de **taille L'**: pour chaque classe, un id de feuille d’origine représentante.

    Retour:
      Z_pruned : np.ndarray, shape (L'−1, 4), dtype float64
      surv_idx : np.ndarray, shape (L',), dtype int64   (un id de feuille d’origine par classe)
    """
    Z = np.asarray(Z_full, dtype=np.float64, order="C")
    t = int(Z.shape[0])
    m = t + 1
    if m <= 1:
        # 0 fusion -> 1 classe, mais on n’a pas de feuille à choisir; on met 0 par convention si m==1
        return np.zeros((0, 4), np.float64), np.array([0], dtype=np.int64)

    left   = Z[:, 0].astype(np.int64,   copy=False)
    right  = Z[:, 1].astype(np.int64,   copy=False)
    weight = Z[:, 2].astype(np.float64, copy=False)
    usize  = np.rint(Z[:, 3]).astype(np.int64, copy=False)

    # Garde SciPy
    for i in range(t):
        p = m + i
        a = int(left[i]); b = int(right[i])
        if not (0 <= a < p and 0 <= b < p):
            raise ValueError(f"Convention SciPy violée à i={i}: enfants {a},{b} >= parent {p}")

    n_nodes = m + t

    # Tailles au moment des fusions: feuilles=K, internes=usize
    size_node = np.zeros(n_nodes, dtype=np.int64)
    size_node[:m] = int(K)
    size_node[m:] = usize

    # Inclusives (ordre indépendant): su == max(sa, sb)
    inc = np.zeros(t, dtype=bool)
    for i in range(t):
        a = int(left[i]); b = int(right[i]); p = m + i
        sa, sb, su = int(size_node[a]), int(size_node[b]), int(size_node[p])
        inc[i] = (su == (sa if sa >= sb else sb))

    # Intervalles de feuilles
    pos, first, last, leaf_order = build_leaf_dfs_intervals(left, right)  # pos: leaf_id -> dfs_idx

    # Union-Find sur les feuilles (indexées par position DFS) + "next pointer" pour unions d’intervalles
    UF = UnionFind(m)           # ← ta version Cython
    nxt = np.arange(m + 1, dtype=np.int64)  # pointeurs de saut

    def _next(i: int) -> int:
        j = i
        while nxt[j] != j:
            nxt[j] = nxt[nxt[j]]
            j = nxt[j]
        return j

    def union_interval(L: int, R: int, to_pos: int):
        root_to = UF.find(to_pos)
        i = _next(L)
        while i <= R:
            UF.union(i, root_to)
            nxt[i] = _next(i + 1)
            i = nxt[i]

    # Contraction: pour chaque inclusion, on fusionne TOUT l’intervalle perdant dans la classe du gagnant
    # L’ordre n’a pas d’importance ici; faire croissant améliore juste la localité mémoire.
    for i in range(t):
        if not inc[i]:
            continue
        a = int(left[i]); b = int(right[i])
        sa, sb = int(size_node[a]), int(size_node[b])
        winner = a if sa >= sb else b
        loser  = b if sa >= sb else a
        Lw, Rw = int(first[winner]), int(last[winner])
        Ll, Rl = int(first[loser]),  int(last[loser])
        rep_pos = UF.find(Lw)
        union_interval(Ll, Rl, rep_pos)

    # Classes sur feuilles (en positions DFS)
    rep_pos = np.array([UF.find(i) for i in range(m)], dtype=np.int64)
    uniq_rep, inverse = np.unique(rep_pos, return_inverse=True)  # uniq_rep: reps de classes, triés
    Lp = int(uniq_rep.size)
    cls_of_pos = inverse  # mapping position DFS -> classe [0..Lp-1]

    # Choix d’un représentant feuille pour CHAQUE classe (ici: la feuille d’origine de plus petite position DFS)
    # surv_idx de taille L' demandé: un id de feuille d’origine par classe
    min_pos_per_class = np.full(Lp, m + 1, dtype=np.int64)
    for p in range(m):
        c = int(cls_of_pos[p])
        if p < min_pos_per_class[c]:
            min_pos_per_class[c] = p
    # convertit position DFS -> leaf_id d’origine via leaf_order
    surv_idx = leaf_order[min_pos_per_class]  # shape (Lp,), dtype int64

    if Lp <= 1:
        return np.zeros((0, 4), np.float64), surv_idx

    # Arêtes pour fusions NON inclusives: relie une classe de gauche à une de droite
    U = []
    V = []
    W = []
    S = []
    for i in range(t):
        if inc[i]:
            continue
        a = int(left[i]); b = int(right[i])
        La = int(first[a]); Lb = int(first[b])  # un représentant de chaque côté suffit
        ca = int(cls_of_pos[UF.find(La)])
        cb = int(cls_of_pos[UF.find(Lb)])
        if ca == cb:
            continue
        if ca > cb:
            ca, cb = cb, ca
        U.append(ca); V.append(cb); W.append(float(weight[i])); S.append(float(usize[i]))

    if not U:
        raise RuntimeError(f"[PRUNE/COMPRESS] Aucune arête non inclusive entre {Lp} classes.")

    U = np.asarray(U, dtype=np.int64); V = np.asarray(V, dtype=np.int64)
    W = np.asarray(W, dtype=np.float64); S = np.asarray(S, dtype=np.float64)

    # Kruskal sur les classes
    UFc = UnionFind(Lp)  # ← ta Cython UF
    comp_id = np.arange(Lp, dtype=np.int64)
    Z_pruned = np.empty((Lp - 1, 4), dtype=np.float64)
    created = 0
    for i in range(U.size):
        ru = UFc.find(int(U[i])); rv = UFc.find(int(V[i]))
        if ru == rv:
            continue
        ida, idb = int(comp_id[ru]), int(comp_id[rv])
        if ida > idb:
            ida, idb = idb, ida
        Z_pruned[created, 0] = float(ida)
        Z_pruned[created, 1] = float(idb)
        Z_pruned[created, 2] = float(W[i])
        Z_pruned[created, 3] = float(S[i])
        UFc.union(ru, rv)
        root = UFc.find(ru)
        comp_id[root] = Lp + created
        created += 1
        if created == Lp - 1:
            break

    if created != Lp - 1:
        raise RuntimeError(f"[PRUNE/COMPRESS] Quotient incomplet: classes={Lp}, edges={created}, attendu {Lp-1}.")

    if verbose:
        print(f"[PRUNE/COMPRESS] feuilles={m}, classes={Lp} | Z_pruned: {Z_pruned.shape}")

    return Z_pruned, surv_idx


def build_Z_mst_occurrences_components(face_vertices,
                                       mst_faces_sorted,    # [(u,v,w)]
                                       min_cluster_size,
                                       verbose=False,
                                       distinct_mode="owner",
                                       DBSCAN_threshold=None):
    def _fmt_bytes(n):
        n = float(n)
        if n < 1024: return f"{n:.0f} B"
        n /= 1024
        if n < 1024: return f"{n:.1f} KiB"
        n /= 1024
        if n < 1024: return f"{n:.1f} MiB"
        n /= 1024
        return f"{n:.2f} GiB"

    def _rss():
        try:
            import psutil
            return psutil.Process().memory_info().rss
        except Exception:
            return None

    face_vertices = np.asarray(face_vertices, dtype=np.int64, order="C")
    n_faces = int(face_vertices.shape[0])
    if n_faces == 0:
        return np.zeros(0, dtype=np.int64)

    # Composantes de FACES via UF sur MST des faces
    UF_face = UnionFind(n_faces)
    for u, v, _ in mst_faces_sorted:
        UF_face.union(u, v)
    comp_labels = np.fromiter((UF_face.find(i) for i in range(n_faces)), count=n_faces, dtype=np.int64)

    # Faces triées par composante
    order_faces = np.argsort(comp_labels, kind='mergesort')
    labels_sorted = comp_labels[order_faces]
    diff_faces = labels_sorted[1:] != labels_sorted[:-1]
    starts = np.r_[0, 1 + np.flatnonzero(diff_faces)]
    ends   = np.r_[starts[1:], labels_sorted.size]
    uniq = labels_sorted[starts]
    faces_ordered = np.arange(n_faces, dtype=np.int64)[order_faces]

    # Préparation du mapping points -> labels
    n_points_total = int(face_vertices.max()) + 1
    labels_points_unique  = np.full(n_points_total, -1, dtype=np.int64)
    labels_points_multiple = [[] for _ in range(n_points_total)]
    first    = np.full(n_points_total, -2, dtype=np.int64)
    conflict = np.zeros(n_points_total, dtype=bool)
    next_cluster_id = 0

    if verbose:
        r = _rss()
        if r is not None:
            print(f"[COMP-F:0] components={uniq.size}, faces={n_faces}, points={n_points_total}, RSS={_fmt_bytes(r)}")

    # Pour chaque composante de FACES, on calcule le Z (faces) et labels de faces, puis on propage aux points
    if len(mst_faces_sorted):
        u_arr = np.asarray([uv[0] for uv in mst_faces_sorted], dtype=np.int64)
        v_arr = np.asarray([uv[1] for uv in mst_faces_sorted], dtype=np.int64)
        w_arr = np.asarray([uv[2] for uv in mst_faces_sorted], dtype=np.float64)
        comp_u = comp_labels[u_arr]
        order_edges = np.argsort(comp_u, kind='mergesort')
        comp_u_sorted = comp_u[order_edges]
        diff_edges = comp_u_sorted[1:] != comp_u_sorted[:-1]
        starts_e = np.r_[0, 1 + np.flatnonzero(diff_edges)]
        ends_e   = np.r_[starts_e[1:], comp_u_sorted.size]
        uniq_e = comp_u_sorted[starts_e]
    else:
        u_arr = v_arr = np.empty(0, dtype=np.int64); w_arr = np.empty(0, dtype=np.float64)
        order_edges = np.empty(0, dtype=np.int64)
        starts_e = ends_e = uniq_e = np.empty(0, dtype=np.int64)

    for j in range(uniq.size):
        f_start, f_end = int(starts[j]), int(ends[j])
        faces = faces_ordered[f_start:f_end]
        if faces.size == 0:
            continue
        comp_id = int(uniq[j])

        # Sous-ensemble d'arêtes du MST des faces pour cette composante
        if order_edges.size:
            pos = np.searchsorted(uniq_e, comp_id)
            if pos < uniq_e.size and uniq_e[pos] == comp_id:
                e_start = int(starts_e[pos]); e_end = int(ends_e[pos])
                idx_edges = order_edges[e_start:e_end]
            else:
                idx_edges = np.empty(0, dtype=np.int64)
        else:
            idx_edges = np.empty(0, dtype=np.int64)

        faces_sorted = np.sort(faces)
        # Remap local
        u_sel = u_arr[idx_edges]; v_sel = v_arr[idx_edges]; w_sel = w_arr[idx_edges]
        new_u = np.searchsorted(faces_sorted, u_sel).astype(np.int64, copy=False)
        new_v = np.searchsorted(faces_sorted, v_sel).astype(np.int64, copy=False)

        faces_compact = face_vertices[faces_sorted]

        if verbose:
            r = _rss()
            est_Z = max(0, faces_compact.shape[0]-1)*4*8
            print(f"[COMP-F:1] comp {j+1}/{uniq.size} | faces={faces_compact.shape[0]}, edges={new_u.size}, "
                  f"Z_est≈{_fmt_bytes(est_Z)}, RSS={_fmt_bytes(r) if r else 'n/a'}")

        # GC au niveau des FACES (plus d’occurrences ici)
        Z_i, labels_faces_i, probabilities_faces_i,  F_i = build_Z_mst_occurrences_gc(
            faces_compact, (new_u, new_v, w_sel),
            min_cluster_size=min_cluster_size, verbose=verbose,
            distinct_mode="owner",
            DBSCAN_threshold=DBSCAN_threshold
        )

        # Propagation labels de faces -> points, avec gestion des conflits
        if len(labels_faces_i) and np.any(labels_faces_i != -1):
            max_local = int(labels_faces_i[labels_faces_i != -1].max())
            offset = next_cluster_id
            for f_loc in range(F_i.shape[0]):
                lbl = int(labels_faces_i[f_loc])
                if lbl == -2 or lbl == -1 :
                    continue
                # if lbl != -1 :
                lbl_g = lbl + offset
                # else :
                #     lbl_g = -1
                # pour chaque sommet de la face f_loc
                row = F_i[f_loc]
                for t in range(row.size):
                    v = int(row[t])
                    labels_points_multiple[v].append((lbl_g, probabilities_faces_i[f_loc]))
                    if first[v] == -2:
                        first[v] = lbl_g
                    elif first[v] != lbl_g:
                        conflict[v] = True
            next_cluster_id = offset + max_local + 1

        if verbose and (j % 1 == 0):
            valid = (first != -2) & (~conflict)
            r = _rss()
            print(f"[COMP-F:2] comp {j+1}/{uniq.size} done | cumul points labellisés={int(valid.sum())} "
                  f"| conflits={int(conflict.sum())} | RSS={_fmt_bytes(r) if r else 'n/a'}")

    # Finalisation des labels Points
    mask_ok = (~conflict) & (first != -2)
    labels_points_unique[mask_ok] = first[mask_ok]
    ret_labels_points_multiple = [[] for _ in range(n_points_total)]
    for v, labels in enumerate(labels_points_multiple):
        dict_labels = {}
        labels_unique = []
        for ind,(lbl,prob) in enumerate(labels) :
            if lbl not in dict_labels :
                dict_labels[lbl] = [ind]
            else :
                dict_labels[lbl].append(ind)
        for lbl,inds in dict_labels.items() :
            size_lbl = len(inds)
            prob_lbl = 0
            for ind in inds :
                prob_lbl += labels[ind][1]
            prob_lbl /= size_lbl
            labels_unique.append((lbl,size_lbl/len(labels),prob_lbl))
        if labels_unique == [] :
            labels_unique = [(-1,1,1)]
        ret_labels_points_multiple[v] = labels_unique

    if verbose:
        uvals = np.unique(labels_points_unique)
        r = _rss()
        print('Clusters finaux :', uvals[uvals != -1].size, '| bruit :', int(np.sum(labels_points_unique == -1)),
              f"| RSS={_fmt_bytes(r) if r else 'n/a'}")

    return labels_points_unique, ret_labels_points_multiple

In [ ]:
# Clone du repo (pas de release/pip officiel)
# !git clone -q https://github.com/geoo89/orderkdelaunay.git


In [ ]:
# import sys, os, pathlib

# repo = pathlib.Path("/content/orderkdelaunay")
# pkg_path = repo / "python"
# assert pkg_path.exists(), "Le dossier 'python' n'a pas été trouvé dans le repo."
# sys.path.insert(0, str(pkg_path))

# # Import de la classe Python (décrite dans le README du dépôt)
# from orderk_delaunay import OrderKDelaunay

# print("Import OK:", OrderKDelaunay)


In [ ]:
# import numpy as np

# # Petit nuage 2D bidon
# rng = np.random.default_rng(0)
# X = rng.random((20, 2))

# # Instancier l’algorithme (API exacte: voir help() ci-dessous)
# okdel = OrderKDelaunay(X,2)
# print("Instance créée.")

# # Découvre l’API dispo dans cette version
# print([name for name in dir(okdel) if "mosaic" in name.lower() or "order" in name.lower()])
# help(OrderKDelaunay)  # scrolle si tu as du courage


In [ ]:
# points = rng.random((10, 2))
# # from plotter import Plotter, Plotter2D, Plotter3D

# # the order k up to which to compute the order-k Delaunay diagram
# order = 3
# # Whether to print the cells of all the complexes
# print_output = True
# # Whether to draw all the order-k Delaunay mosaics
# draw_output = True

# # Compute the order-k Delaunay mosaics
# orderk_delaunay = OrderKDelaunay(points, order)

# # Initialize appropriate plotter for drawing the mosaics.
# dimension = len(points[0])
# if dimension == 2:
#     plotter = Plotter2D(points, orderk_delaunay)
# elif dimension == 3:
#     plotter = Plotter3D(points, orderk_delaunay)
# else:
#     # Stub that doesn't draw anything.
#     plotter = Plotter(points, orderk_delaunay)

# for k in range(1, order+1):
#     if draw_output:
#         plotter.draw(k)

#     if print_output:
#         cells = orderk_delaunay.diagrams_cells[k-1]
#         # Output all the cells.
#         print("Order {}. Number of cells: {}".format(
#                 len(cells[0][0]), len(cells)))
#         print(len(orderk_delaunay.diagrams_simplices[k-1]), orderk_delaunay.diagrams_vertices[k-1])
#         for cell in sorted(cells):
#             print(cell)

In [ ]:
import math, sys
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from itertools import combinations
from operator import itemgetter
!pip -q install MiniballCpp
import miniball  # oui, le nom est malheureux = cyminiball
# import importlib, importlib.metadata as im



def _kth_radius(M,k,metric,pre):
    if pre:
        return np.partition(M,k,axis=1)[:,k]
    nn=NearestNeighbors(n_neighbors=k+1,metric=metric).fit(M)
    d,_=nn.kneighbors(M)
    return d[:,k]

def MEB(points_sub):
    """
    Centre, Rayon^2 MEB d'un sous-ensemble de points (k+1, d).
    Utilise miniball. Retourne 0. si un seul point.
    """
    if points_sub.shape[0] <= 1:
        return points_sub[0], 0.0
    if points_sub.shape[0] == 2:
        # edge: (||p-q||/2)^2
        diff = points_sub[0] - points_sub[1]
        return 0.5 * (points_sub[0] + points_sub[1]), float(np.dot(diff, diff))*0.25
    # k>=2
    mb = miniball.Miniball(points_sub)
    center, r2 = mb.center(), mb.squared_radius()
    return center, r2



# complex = \in ["auto", "orderk_delaunay", "delaunay", "rips"]
# precision in ["safe", "exact"]
def _build_graph_KSimplexes(M,K,min_samples,metric,complex_chosen,expZ,precision="safe",verbeux=False) :
    if min_samples is None or min_samples <= K :
        min_samples = K+1
    pre = metric=='precomputed'
    Delaunay_possible = not pre and metric=='euclidean' and M.ndim == 2 and M.shape[0] != M.shape[1]
    n,d = M.shape[0],M.shape[1]
    if complex_chosen.lower() not in ["orderk_delaunay", "delaunay", "rips"] : # "auto"
        if not Delaunay_possible :
            complex_chosen = "rips"
        else :
            if d > 10 and n > 100 :
                complex_chosen = "rips"
            elif d > 10 :
                complex_chosen = "delaunay"
            elif d > 5 and n > 1000 :
                complex_chosen = "rips"
            else :
                complex_chosen = "orderk_delaunay"
    Simplexes = []
    if complex_chosen.lower() == "orderk_delaunay" :
        Simplexes_sans_f = []
        if d >= 3 or True :
            Simplexes_sans_f = orderk_delaunay3(M, min_samples-1, precision=precision, verbeux=verbeux)
            if verbeux :
                print(f"Simplexes sans filtration : {len(Simplexes_sans_f), Simplexes_sans_f[:5]}")
        else :
            set_simplexes = set()
            orderk_delaunay = OrderKDelaunay(M, min_samples-1,verbeux)
            cells = orderk_delaunay.diagrams_cells[min_samples-1-1]
            for cell in cells :
                for s1,s2 in itertools.combinations(cell,2) :
                    vertices_set = set()
                    for p in s1 :
                        vertices_set.add(int(p))
                    for p in s2 :
                        vertices_set.add(int(p))
                    vertices = list(sorted(vertices_set))
                    simplexes = itertools.combinations(vertices, K+1)
                    for simplexe in simplexes :
                        simplexe = tuple(sorted(simplexe))
                        if not simplexe in set_simplexes :
                            set_simplexes.add(simplexe)
                            simplexe = list(sorted(simplexe))
                            Simplexes_sans_f.append(simplexe)
        def _sqr_radius_for_simplex(simplex):
            idx = np.asarray(simplex, dtype=np.int64)        # indices 1D vers les points de M
            P = M[idx]                                      # petite copie (d+1, d) → ok
            return miniball.Miniball(P).squared_radius()
        # !pip -q install joblib
        from joblib import Parallel, delayed
        radii_sq = Parallel(n_jobs=N_cpu, prefer="processes")(delayed(_sqr_radius_for_simplex)(s) for s in Simplexes_sans_f)
        # radii_sq = Parallel(n_jobs=N_cpu, prefer="processes", batch_size=max(1, len(Simplexes_sans_f)//64), max_nbytes="256M")(delayed(lambda A, s: miniball.Miniball(A[s]).squared_radius())(M, s) for s in Simplexes_sans_f)
        if expZ != 2 :
            radii_sq = np.asarray(radii_sq)
            radii_sq = radii_sq ** (expZ / 2)
        Simplexes = [(s,radii_sq[ind]) for ind,s in enumerate(Simplexes_sans_f)]

        # ch = max(1, len(Simplexes)//(N_cpu*8))   # ~64 paquets
        # with ProcessPoolExecutor(max_workers=N) as ex:
        #     radii_sq = list(ex.map(_sqr_radius_for_simplex, Simplexes, chunksize=ch))
    else : # "delaunay" or "rips"
        import gudhi
        r = _kth_radius(M,min_samples-1,metric,pre)
        r2 = r**2
        if complex_chosen.lower() == "rips" :
            r2 = r
            expZ *= 2
            if precision == "exact" :
                mx=2*np.quantile(r,0.99)
            else : # "safe"
                mx=(1 + 1/np.sqrt(d))*np.quantile(r,0.99)
            if pre or metric!='euclidean':
                D=M if pre else pairwise_distances(M,metric=metric)
                st=gudhi.RipsComplex(distance_matrix=D,max_edge_length=mx).create_simplex_tree(max_dimension=K)
            else: # petite optimisation : on ne construit que jusqu'à la dimension K-1
                st=gudhi.RipsComplex(points=M,max_edge_length=mx).create_simplex_tree(max_dimension=K)
        else : # complex_chosen.lower() == "delaunay" :
            st = gudhi.DelaunayCechComplex(points=M).create_simplex_tree()
        for s, f in st.get_skeleton(K):
            if len(s) != K + 1:
                continue
            simplexe = list(sorted(s))
            max_kth_radius2 = 0
            for p in s:
                max_kth_radius2 = max(max_kth_radius2, r2[p])
            f = max(f, max_kth_radius2)
            if expZ != 2:
                f = f ** (expZ / 2)
            Simplexes.append((simplexe,f))
    faces_raw = []
    e_u, e_v, e_w = [], [], []
    nS = 0
    for s, f in Simplexes :
        if len(s) <= K :
            continue
        for KP1_items_choisis in itertools.combinations(range(len(s)), K+1) :
            nS += 1
            vert = tuple(sorted(KP1_items_choisis))
            base = len(faces_raw)
            for drop in range(K + 1):
                faces_raw.append([s[vert[i]] for i in range(K + 1) if i != drop])
            for k in range(K):
                e_u.append(base + k)
                e_v.append(base + k + 1)
                e_w.append(float(f))
    return faces_raw, e_u, e_v, e_w, nS


def HypergraphPercol(M, K=2, min_cluster_size=None, min_samples=None, metric="euclidean", DBSCAN_threshold=None, label_all_points=False, return_multi_clusters=False, complex_chosen="auto", expZ=2, precision="safe", dim_reducer=False, threshold_variance_dim_reduction=0.999, verbeux=False):
    n,d=M.shape[0],M.shape[1]
    M = np.ascontiguousarray(M, dtype=np.float64)
    if min_cluster_size is None :
        min_cluster_size = round(np.sqrt(n))
    X = np.copy(M)
    pre = metric=='precomputed'
    Delaunay_possible = not pre and metric=='euclidean' and M.ndim == 2 and M.shape[0] != M.shape[1]
    if min_samples is None or min_samples <= K :
        min_samples = K+1
    if str(dim_reducer).lower() in ['pca', 'umap'] and Delaunay_possible:
        pca = PCA(n_components=threshold_variance_dim_reduction, svd_solver="full", whiten=False)
        X2 = pca.fit_transform(M)
        r  = pca.n_components_                     # dimension retenue
        ratio = pca.explained_variance_ratio_.sum()  # variance conservée (≈ 0.999)
        if r < d :
            if dim_reducer.lower() == 'pca' :
                X = X2
                if verbeux :
                    print("Dimension réduite par Analyse en Composantes Principales (PCA) :\n d → r : ",d,"→",r," Variance conservée :",ratio)
            else : # 'umap'
                from umap import UMAP
                reducer = UMAP(n_components=r,n_neighbors=max(2*2*(K+1),min_samples),metric=metric)#,random_state=0,)
                X = reducer.fit_transform(M)
                if verbeux :
                    print("Dimension réduite par UMAP :\n d → r : ",d,"→",r)


    faces_raw, e_u, e_v, e_w, nS = _build_graph_KSimplexes(X,K,min_samples,metric,complex_chosen,expZ,precision,verbeux)
    if verbeux:
        print(f"{K}-simplices={nS}")

    if not faces_raw:
        if K > d :
            print("Warning: K too high compared to the dimension of the data. No clustering possible with such a K.")
        if return_multi_clusters :
            return np.full(n, -1, dtype=np.int64), [(-1,1,1)]*n
        else :
            return np.full(n, 0, dtype=np.int64) ## ATTENTION

    faces_raw = np.asarray(faces_raw, dtype=np.int64, order="C")
    e_u = np.asarray(e_u, dtype=np.int64)
    e_v = np.asarray(e_v, dtype=np.int64)
    e_w = np.asarray(e_w, dtype=np.float64)

    # [3] Dédup des faces + inverse
    faces_unique, inv = np.unique(faces_raw, axis=0, return_inverse=True)
    if verbeux:
        print(f"Faces uniques: {faces_unique.shape[0]} (compression {faces_raw.shape[0]}→{faces_unique.shape[0]})")

    u = inv[e_u]; v = inv[e_v]; w = e_w

    # [4] Dédup des arêtes (min poids) en vectoriel
    uu = np.minimum(u, v); vv = np.maximum(u, v)
    order = np.lexsort((vv, uu))
    uu = uu[order]; vv = vv[order]; ww = w[order]
    change = np.r_[True, (uu[1:] != uu[:-1]) | (vv[1:] != vv[:-1])]
    gidx = np.flatnonzero(change)
    ww = np.minimum.reduceat(ww, gidx)
    uu = uu[gidx]; vv = vv[gidx]
    if verbeux:
        print(f"Arêtes uniques (u<v): {uu.size} (avant dédup {u.size})")

    # [5] MST des faces via Kruskal + UnionFind
    UFf = UnionFind(faces_unique.shape[0])
    mst_faces_sorted = _kruskal_mst_from_edges(faces_unique.shape[0], uu, vv, ww, UFf)
    if verbeux:
        m = faces_unique.shape[0]; e_mst = len(mst_faces_sorted)
        comps = max(0, m - e_mst) if m else 0
        print(f"MST faces: {e_mst} arêtes, composantes estimées: {comps}")

    # 6) Clustering via composants/occurrences optimisés (fonctions existantes inchangées ailleurs)
    labels_points_unique, labels_points_multiple = build_Z_mst_occurrences_components(
        faces_unique, mst_faces_sorted,
        min_cluster_size=min_cluster_size,
        verbose=verbeux,
        distinct_mode="owner",
        DBSCAN_threshold=DBSCAN_threshold
    )

    for i in range(n) :
        labels_i = labels_points_multiple[i]
        if len(labels_i) > 0 :
            for label, prop, prob, *_ in labels_i :
                if prop > 1/2 and prob > 1/2 :
                    labels_points_unique[i] = label
                    break
    labels_points_unique = np.asarray(labels_points_unique)

    def knn_fill_weighted(X, labels, k):
        from sklearn.neighbors import KNeighborsClassifier
        X = np.asarray(X); y = np.asarray(labels).copy()
        mask_u = (y == -1);
        if not mask_u.any(): return y
        mask_l = ~mask_u
        if not mask_l.any(): return y
        k = min(k, mask_l.sum())
        clf = KNeighborsClassifier(n_neighbors=k, weights="distance", n_jobs=-1)
        clf.fit(X[mask_l], y[mask_l])
        y[mask_u] = clf.predict(X[mask_u])
        return y

    if label_all_points and Delaunay_possible :
        labels_points_unique = knn_fill_weighted(M, labels_points_unique, min_samples)

    if return_multi_clusters :
        return labels_points_unique, labels_points_multiple
    else :
        return labels_points_unique

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
